In [125]:
import numpy as np
import open3d as o3d
import os
import pandas as pd
from tqdm import tqdm
import pickle
from collections import defaultdict

o3d.utility.set_verbosity_level(o3d.cpu.pybind.utility.VerbosityLevel(0))

In [11]:
pokedex = pd.read_csv('E:\\3d-pokemon-generator\data\Pokedex_Ver_SV2.csv') # path to your csv file
pokedex = pokedex[['Original_Name', 'Type1', 'Type2']].drop_duplicates('Original_Name').reset_index(drop=True)

In [95]:
dirs = []
names = []
types = []

for root_dir, cur_dir, files in os.walk(r'E:\3d-pokemon-generator\data\modele_pokemon'): # path to your modele_pokemon folder
    for file in files:
        if file.endswith('.obj') or file.endswith('.OBJ'):
            for idx, name in enumerate(pokedex['Original_Name']):
                if name.lower() in root_dir.lower():
                    names.append(name)
                    types.append(pokedex.loc[idx][1])
                    dirs.append(root_dir + '\\' + file)

                    if str(pokedex.loc[idx][2]) != 'nan':
                        names.append(name)
                        types.append(pokedex.loc[idx][2])
                        dirs.append(root_dir + '\\' + file)

                    break

In [138]:
dataset = pd.DataFrame(columns=['Name', 'Type', 'Points_coordinates'])

for sample_idx in tqdm(range(len(names))):
    try:
        textured_mesh = o3d.io.read_triangle_mesh(dirs[sample_idx])
        pcd = textured_mesh.sample_points_uniformly(number_of_points=20000)
        pure_array = np.array(pcd.points)

        dataset.loc[sample_idx] = [names[sample_idx], types[sample_idx], pure_array]
    except RuntimeError:
        pass

dataset.reset_index(inplace=True, drop=True)

100%|██████████| 4885/4885 [01:52<00:00, 43.35it/s]


In [139]:
dataset

,Name,Type,Points_coordinates
0,Squirtle,Water,"[[0.9300414219934687, 3.863947015429417, 2.252..."
1,Pikachu,Electric,"[[-7.001530540729028, 18.34376707098572, 1.859..."
2,Pikachu,Electric,"[[0.0029545852310238737, 20.549869980770723, 7..."
3,Pikachu,Electric,"[[-0.14823590880304008, 20.70945511690288, 7.9..."
4,Pikachu,Electric,"[[-0.09233796966611395, 6.18907922519907, 2.84..."
...,...,...,...
4880,Panpour,Water,"[[4.129633589832427, 49.915269048591966, 1.861..."
4881,Simipour,Water,"[[-56.27355224922986, 60.295405577985, 13.2092..."
4882,Woobat,Psychic,"[[8.954510301072329, 19.588021050082762, 18.15..."
4883,Woobat,Flying,"[[8.94283310289448, 19.60570145736483, 18.1798..."


In [140]:
dataset.to_pickle('../data/preprocessed_samples/4877.pickle')

In [141]:
dataset = pd.read_pickle('../data/preprocessed_samples/4877.pickle')